# Open-Closed Principle

> When you add a functionality, you should add it by extension rather than by modification > open for extension, closed for modification.

Let's define a Product class and some attributes that a product may have, such as Color and Size.

In [2]:
from enum import Enum

class Color(Enum):
    RED = 1
    GREEN = 2
    BLUE = 3

class Size(Enum):
    SMALL = 1
    MEDIUM = 2
    LARGE = 3

class Product:
    def __init__(self, name, color, size):
        self.name = name
        self.color = color
        self.size = size

Now we want to be able to filter by product attributes.

In [3]:
class ProductFilter:
    def filter_by_color(self, products, color):
        for p in products:
            if p.color == color: yield p

    def filter_by_size(self, products, size):
        for p in products:
            if p.size == size: yield p

    def filter_by_size_and_color(self, products, size, color):
        for p in products:
            if p.color == color and p.size == size:
                yield p

    # we could keep adding boolean ops and additional conditions...

This doesn't scale at all! If we had 3 or more features, the amount of methods would explode!

> This is called the State Space Explosion.

You would also be violating OCP each time you added a new feature to your products because you'd have to add new filters to this class.

We will now use the [Specification design pattern](https://en.wikipedia.org/wiki/Specification_pattern) to make our solution scalable and keep the Open-Closed Principle. 

We will create a Specification base class that contains a single `is_satisfied() -> Bool` method that will return `True` if a condition is met, and we'll create separate `FeatureSpecifications` classes for each feature that will implement that method.

In [4]:
class Specification:
    """Base class"""
    def is_satisfied(self, item):
        """All classes that inherit from Specification must implement this method"""
        raise NotImplementedError()

    def __and__(self, other):
        """
        Overloads the ampersand (&) operator, allowing us to use AND operations with different Specification objects.
        We overload the & operator because we cannot overload the "and" keyword.
        Check below for the AndSpecification code
        """
        return AndSpecification(self, other)

We will now create a Filter class. Its method will simply pass, because the idea is that you extend by inheriting from other classes and add functionality there.

This is how you avoid modifying already existing classes!

In [5]:
class Filter:
    """Seems redundant; see explanation below"""
    def filter(self, items, spec):
        pass

class BetterFilter(Filter):
    def filter(self, items, spec):
        for item in items:
            if spec.is_satisfied(item):
                yield item

In the example above, even though the base class `Filter` doesn't seem to do anything and is unnecesary for `BetterFilter`, we're preserving OCP because `BetterFilter` is making assumptions about the implementation, such as assumning that `items` is an iterable object. There could be some instances where this is not the case; you could then create a separate class that inherits from `Filter` and implements the proper business logic for that particular use case.

Now, let's create our feature specifications.

In [6]:
class ColorSpecification(Specification):
    def __init__(self, color):
        self.color = color

    def is_satisfied(self, item):
        return item.color == self.color

class SizeSpecification(Specification):
    def __init__(self, size):
        self.size = size

    def is_satisfied(self, item):
        return item.size == self.size

class AndSpecification(Specification):
    """Combinator class"""
    def __init__(self, *args):
        self.args = args

    def is_satisfied(self, item):
        """Explanation below"""
        return all(map(
            lambda spec: spec.is_satisfied(item), self.args))

Note that `AndSpecification` is a *Combinator class*; a Combinator is a structure that combines other structures.

For this example, where we only combine a maximum of 2 classes, we could implement the Combinator in a more straight-forward manner, but the way it's implemented here makes it more generic and able to handle any number of classes:
* The `__init__` method can get any number of arguments, which are then stored as a list in `self.args`.
* The `is_satisfied` method will go through all the stored args and make sure that they're all satisfied.
    * `map()` goes through every single element of `self.args` and applies a lambda function to each element.
    * The lambda function takes a specification (a single argument from `self.args`) and checks that it's satisfied for the `item` we're passing as a parameter.
    * `all()` checks that every single element in the map (the returns of all the lambdas) is `True`.

Now let's create our sample products and product list:

In [7]:
apple = Product('Apple', Color.GREEN, Size.SMALL)
tree = Product('Tree', Color.GREEN, Size.LARGE)
house = Product('House', Color.BLUE, Size.LARGE)

products = [apple, tree, house]

This is how we would filter using the old/bad filter:

In [8]:
pf = ProductFilter()

print('Green products (old):')
for p in pf.filter_by_color(products, Color.GREEN):
    print(f' - {p.name} is green')

Green products (old):
 - Apple is green
 - Tree is green


Let's filter now using our new and improved filter that respects OCP. We will instantiate our specifications as needed and use `BetterFilter.filter()` for filtering all the features, without requiring separate filter methods for each feature. Also, we can use our overloaded `&` operator to instantiate an `AndSpecification` object by combining 2 specifications.

In [9]:
bf = BetterFilter()

print('Green products (new):')
green = ColorSpecification(Color.GREEN) # ColorSpecification for Green
for p in bf.filter(products, green):
    print(f' - {p.name} is green')

print('Large products:')
large = SizeSpecification(Size.LARGE) # SizeSpecification for Large
for p in bf.filter(products, large):
    print(f' - {p.name} is large')

print('Large blue items:')
# We instantiate an AndSpecification of Large and Blue using our & overloading
large_blue = large & ColorSpecification(Color.BLUE)
for p in bf.filter(products, large_blue):
    print(f' - {p.name} is large and blue')

Green products (new):
 - Apple is green
 - Tree is green
Large products:
 - Tree is large
 - House is large
Large blue items:
 - House is large and blue
